In [1]:
"""
Purpose: To pull down the neuron split suggestions
from the datajoint table and then write them to a spreadsheet

"""

'\nPurpose: To pull down the neuron split suggestions\nfrom the datajoint table and then write them to a spreadsheet\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru
import preprocessing_vp2 as pre

INFO - 2021-01-12 18:55:45,908 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-12 18:55:45,909 - settings - Setting database.user to celiib
INFO - 2021-01-12 18:55:45,910 - settings - Setting database.password to newceliipass
INFO - 2021-01-12 18:55:45,934 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-12 18:55:45,935 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:55:45,947 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-12 18:55:46,260 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:55:55,082 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:55:55,418 - settings - Setting enable_python_native_blobs to True


# Downloading the limb results

In [6]:
minnie.NeuronSplitSuggestions()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,split_results will store the results of how to split the limbs of neuron objects,run_time the amount of time to run (seconds)
864691134947393276,0,0.25,29.00,0,=BLOB=,100.245
864691135012471798,0,0.25,29.00,0,=BLOB=,342.8413
864691135012571126,0,0.25,29.00,0,=BLOB=,217.0619
864691135012905110,0,0.25,29.00,0,=BLOB=,154.318
864691135065032772,0,0.25,29.00,0,=BLOB=,121.376
864691135065033540,0,0.25,29.00,0,=BLOB=,182.2972
864691135081745143,0,0.25,29.00,0,=BLOB=,93.8166
864691135155574372,0,0.25,29.00,0,=BLOB=,230.5511
864691135210051520,0,0.25,29.00,0,=BLOB=,74.0144
864691135274155493,0,0.25,29.00,0,=BLOB=,207.4935


In [7]:
split_suggestions_data = minnie.NeuronSplitSuggestions.fetch(as_dict=True)

# Converting the Data to links

In [11]:
import proofreading_utils as pru

In [85]:
spreadsheet_data = []

import re
def ipython_html_object_to_link(html_obj):
    links = re.findall("href=[\"\'](.*?)[\"\']", html_obj.data)
    return links[0]

for curr_data in split_suggestions_data:
    curr_link = pru.split_info_to_neuroglancer_link(segment_id=curr_data["segment_id"],
                                        split_info = curr_data["split_results"],
                                        output_type="local"
                                       )
    
    curr_link_html = ipython_html_object_to_link(curr_link)

    n_suggested_cuts = len(pru.get_all_coordinate_suggestions(curr_data["split_results"]))
    n_paths_not_cut = pru.get_n_paths_not_cut(curr_data["split_results"])

    print(f"n_suggested_cuts = {n_suggested_cuts}, n_paths_not_cut = {n_paths_not_cut}")

    local_dict = dict(segment_id=curr_data["segment_id"],
                     n_suggested_cuts=n_suggested_cuts,
                     n_paths_not_cut=n_paths_not_cut,
                     link=curr_link_html)
    
    spreadsheet_data.append(local_dict)

import pandas as pd
allen_spreadsheet = pd.DataFrame.from_dict(spreadsheet_data)
allen_spreadsheet.to_csv("allen_spreadsheet.csv", sep=',')

Number of different layers = 3
n_suggested_cuts = 0, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 6, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 4, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 3, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 0, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1

# Debugging the split info 

In [69]:
import matplotlib_utils as mu
import pandas as pd
from annotationframeworkclient import FrameworkClient
from nglui import statebuilder
import copy

In [83]:
segment_id = curr_data["segment_id"]
split_info=curr_data["split_results"]
split_coordinates = None
cut_path_coordinates = None
not_cut_path_coordinates = None
other_annotations = dict()
output_type = "local"
split_coordinates_color = "red"
split_coordinates_name = "split_location"
cut_path_coordinates_color = "white"
cut_path_coordinates_name = "cut_path"
not_cut_path_coordinates_color = "green"
not_cut_path_coordinates_name = "not_cut_path"
verbose = True

if split_coordinates is None:
    split_coordinates = pru.get_all_coordinate_suggestions(split_info)

if cut_path_coordinates is None or not_cut_path_coordinates is None:
    cut_path_coordinates,not_cut_path_coordinates = pru.get_all_cut_and_not_cut_path_coordinates(split_info)


annotations_info = {
                    cut_path_coordinates_name:dict(coordinates=cut_path_coordinates,
                                               color=cut_path_coordinates_color),
                    not_cut_path_coordinates_name:dict(coordinates=not_cut_path_coordinates,
                                               color=not_cut_path_coordinates_color),
                    split_coordinates_name:dict(coordinates=split_coordinates,
                                               color=split_coordinates_color),

                   }

annotations_info.update(other_annotations)

df_list = []
states_list = []

for ann_name,ann_dict in annotations_info.items():
    sb = pru.set_state_builder(layer_name=ann_name,
                                     color=mu.color_to_hex(ann_dict["color"]))
    df = pru.set_edit_dataframe(ann_dict["coordinates"],segment_id)

    states_list.append(sb)
    df_list.append(df)

if verbose:
    print(f"Number of different layers = {len(states_list)}")

# building the chained state
chained_sb = statebuilder.ChainedStateBuilder(states_list)

if output_type=="local":
    return_value= chained_sb.render_state(df_list, return_as='html')
elif output_type == "server":
    # To upload to the state server for a shortened URL:
    client = pru.get_client()
    state = chained_sb.render_state(df_list, return_as='dict')
    state_id = client.state.upload_state_json(state)
    short_url = client.state.build_neuroglancer_url(state_id, ngl_url=client.info.viewer_site())
    return_value = short_url
else:
    raise Exception("Not implemented")

Number of different layers = 3


In [75]:
df_list[1]

,split_location,root_id,priority
0,"[[210333.75, 110171.75, 16456.15], [211534.25,...",864691135428492848,1


In [81]:
a,b = pru.get_all_cut_and_not_cut_path_coordinates(split_info)

In [82]:
b

[array([210333.75, 110171.75,  16456.15]),
 array([211534.25 , 101889.5  ,  15706.325]),
 array([212255.25 , 124903.   ,  16450.675]),
 array([213137.06666667, 132676.39166667,  16475.19833333]),
 array([213490.95462904, 130456.04436984,  16470.43199707]),
 array([216020.75 ,  94271.25 ,  15536.025]),
 array([230220.5  ,  95348.25 ,  17260.175]),
 array([233598.75 , 100353.25 ,  17561.425]),
 array([244763.25, 114953.75,  18395.4 ]),
 array([247279.5  , 131543.   ,  18759.925]),
 array([248378. , 147034.5,  19011.7]),
 array([248457.5 , 150756.75,  19116.9 ]),
 array([251002.5 , 173051.5 ,  19329.35]),
 array([251757.5 , 175771.  ,  19400.15]),
 array([252380.   , 178798.25 ,  19447.275]),
 array([252672.5  , 179454.5  ,  19450.875]),
 array([254065. , 183900. ,  19636.4]),
 array([256025.   , 192415.5  ,  19726.975]),
 array([256190. , 189533.5,  19694.4])]